## 環境変数読み込み
- OPENAI_API_KEY
- LANGCHAIN_TRACING_V2
- LANGCHAIN_API_KEY
- LANGCHAIN_PROJECT: ログはProjectごとに保存される。
- TAVILY_API_KEY: AIエージェント専用に構築された検索エンジン。

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

## ログを飛ばすプロジェクト
ログを飛ばすプロジェクトは、環境変数 `LANGCHAIN_PROJECT` で指定する。

In [2]:
import os
print(os.environ["LANGCHAIN_PROJECT"])

machine-learning-workshop


## Quick Start

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
llm.invoke("Hello, world!")

AIMessage(content='Hello! How can I assist you today?')

## LangChain Agentのログ

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults

tool = TavilySearchResults()

In [5]:
tool.invoke({"query": "直近の株価"})

[{'url': 'https://finance.yahoo.co.jp/quote/998407.O',
  'content': '998407.O 20分ディレイ株価 15:15 年初来高値 日経平均株価 38,487.24 ポートフォリオに追加 前日比 +329.30 (+0.86 %) 詳細情報 チャート 時系列 ニュース'},
 {'url': 'https://www.nikkei.com/markets/worldidx/chart/nk225/',
  'content': '日経平均株価 日経の指数について（指数公式ページ） 採用銘柄一覧 38,157.94 (15日 15:15 大引) ＋454.62 (＋1.21%) (単位:円) 2/1 1/1 12/1 32,000 33,000 34,000 35,000 36,000 37,000 38,000 0 2,000,000 4,000,000 6,000,000 表示期間 1日 3ヵ月 6ヵ月...'},
 {'url': 'https://nikkei225jp.com/chart/',
  'content': '採用銘柄は毎年見直される他、臨時に入れ替えがなされることもあります。権利落ちに対する株価修正を行い連続性を保っています。一方、単純平均の為に値がさ株（株価が高い株）の影響力が大きくなります。ハイテク企業や金融企業に値がさ株が多いため日経平均はハイテク株や金融株の動きに左右されやすいという特徴を持ってます。\nリンクフリーです。他サイトへのリンクは新しいウィンドウで開きます。\n・夜間の日経先物動向にも注目 ⇒ 日経先物CME\n・日経平均株価に大きな影響を与える銘柄をチェック ⇒ 日経平均 寄与度\n・どの業種が大きく変動しているのか？業種別株価指数をチェック ⇒ 業種別株価指数\n・日経平均にもっとも影響を与えるドル円 為替をチェック ⇒ 為替 ドル円\n・日経平均株価に影響を与えるNY市況をチェック ⇒ NYダウ\n・世界の株価や為替、先物の情報を全体的にチェック ⇒ ch225 世界の株価 タブレット\n・スマホやiPhoneでも日経平均株価をチェックしたい ⇒ 225225.jp 世界の株価 スマホ\n世界の株価\n日経先物CME\n日経平均\nアジア株価\nヨーロ

In [6]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent

base_prompt = hub.pull("langchain-ai/openai-functions-template")

In [7]:
base_prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input', 'instructions'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['instructions'], template='{instructions}')), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), Mes

In [8]:
instructions = """You are an assistant."""
prompt = base_prompt.partial(instructions=instructions)

In [9]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'instructions': 'You are an assistant.'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['instructions'], template='{instructions}')), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_v

In [10]:
llm = ChatOpenAI(temperature=0)
tavily_tool = TavilySearchResults()
tools = [tavily_tool]
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)

In [11]:
agent_executor.invoke({"input": "直近の日銀の発言は円安と円高のどちらに働くか"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': '直近の日銀の発言は円安と円高のどちらに働くか'}`


[{'url': 'https://globe.asahi.com/article/15009715', 'content': 'わかりやすく簡単に解説：朝日新聞GLOBE＋ 円安・円高とはどういう状態？ わかりやすく簡単に解説 World Now 更新日：2023.09.27 公開日：2023.09.27 2023年4月までの10年間にわたり、日本銀行総裁を務めた黒田東彦氏。 就任後間もなく、日本経済のデフレ脱却のため「2％インフレ目標を2年で実現する」と宣言。...'}, {'url': 'https://www.bbc.com/japanese/features-and-analysis-63422147', 'content': '政府・日銀による円買い介入は、日本の金融危機の最中に円安が進んだ1998年6月以来、24年3カ月ぶりだった。 この介入は短期間は効果があったものの、日本円は再び下落を続け、20日には1ドル＝150円台にまで値下がりした。 Getty Images'}, {'url': 'https://www3.nhk.or.jp/news/html/20231101/k10014243841000.html', 'content': '注目 円安加速 その要因は？ 先月に入って円相場は1ドル＝150円前後で推移していましたが、10月31日、日銀が金融政策の運用をさらに柔軟化すると決めたことをきっかけに海外の投資家を中心とした円売りが膨らみ円安が加速しました。 円安が進んだ背景には2つの要因があります。...'}, {'url': 'https://www.asahi.com/articles/ASS2913WGS28ULFA038.html', 'content': '8日の外国為替市場で円安ドル高が進み、一時、約2カ月半ぶりに1ドル=149円台まで下落した。日本銀行の内田真一副総裁が、大規模な金融緩和から ...'}, {'url': 'https://www.bloomberg.co.jp/ne

{'input': '直近の日銀の発言は円安と円高のどちらに働くか',
 'output': '直近の日銀の発言によると、円高に働く可能性が高いとされています。日本銀行が金融政策の運用を柔軟化することで、海外の投資家を中心とした円売りが増加し、円安が加速しています。また、市場関係者やエコノミストたちは、日本銀行がマイナス金利政策から脱却し、海外の主要中銀が利下げに向かうことで、来年は円高が進むと予想しています。'}